# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [1]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/user/Documents/HEP/VBF-Analysis/VBFAnalysisPlots/TauTau/TauhadTaulep/High-Mass/"
SR = "NewBDTTightTauOS/"
SRSS = "NewBDTTightTauSS/"
CR = "NewBDTMJOS/"
CRSS = "NewBDTMJSS/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Ztautau_SherpaRW"

RQCDwithCRs = False
CalculateRQCD = False

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if ('.root' in i and 'Wjets' not in i)]
print(samples)
if "MJ.root" in samples and not CalculateRQCD:
    os.system("rm "+channelPath+SR+"MJ.root")

Welcome to JupyROOT 6.30/02
['MJ.root', 'Ztautau_MGNLORW.root', 'Ztautau_SherpaRW.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'W_EWK_PoPy.root', 'Ztautau_SherpaNLORW.root', 'Signal_PoPy.root', 'Data.root', 'MC.root', 'VV.root', 'Higgs.root', 'VV_EWK.root', 'W_EWK_Sherpa.root', 'ttbar.root', 'singletop.root']


In [2]:
import sys
sys.path.append('/Users/user/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import tautauInclusiveHistograms,tautauHighMassHistograms,tautauHighMassMJHistograms,emuHighMassHistograms,tautauZpeakHistograms,tautauHiggsBDTHistograms,tautauHiggsHistograms, tautauHighMassCutBasedHistograms, tautauHighMassTightTauHistograms
histos = tautauHighMassTightTauHistograms

In [3]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['VV',"ttbar",'singletop']#+['Wjets',]
if ("Tau" in channelPath) or ("MuEle" in channelPath):
    backgroundSamples += ['Higgs','Zjets','W_EWK_Sherpa','VV_EWK']
mcSamples += backgroundSamples

print('Using the following samples to perform the data subtraction: \n',mcSamples)

# First evaluate if there is evidence for MJ BG.
totalMJisZero = False
nBJetsHistogram = dataSubtract("n_bjets",channelPath+SRSS,"Data",mcSamples,histos,rebin=False)
totalMJ = nBJetsHistogram.GetBinContent(1)
totalMJUncertainty = nBJetsHistogram.GetBinError(1)
if (totalMJ < 0.0) or (totalMJ/totalMJUncertainty < 1.0 ):
    totalMJisZero = True
    print("NO MJ EVIDENCE!")
print("MJ = ",totalMJ," +- ",totalMJUncertainty)

if totalMJ < 0.0:
    totalMJ = 0.0

def errorAoverB (histoA,histoB):
    A = histoA.GetBinContent(1)
    B = histoB.GetBinContent(1)
    AError = histoA.GetBinError(1)
    BError = histoB.GetBinError(1)
    return (A/B)*np.sqrt((AError/A)**2+(BError/B)**2)
    


Using the following samples to perform the data subtraction: 
 ['Signal_Sherpa', 'Ztautau_SherpaRW', 'VV', 'ttbar', 'singletop', 'Higgs', 'Zjets', 'W_EWK_Sherpa', 'VV_EWK']
NO MJ EVIDENCE!
MJ =  0.7664051055908203  +-  2.046623661919995


# Calculate RQCD

In [4]:
# Calculate RQCD in MJCRs
import ctypes
def errorAoverB (histoA,histoB,binRange):
    AErrorD = ctypes.c_double()
    BErrorD = ctypes.c_double()
    A = histoA.IntegralAndError(binRange[0],binRange[1],AErrorD)
    B = histoB.IntegralAndError(binRange[0],binRange[1],BErrorD)
    AError = AErrorD.value
    BError = BErrorD.value
    
    return (A/B)*np.sqrt((AError/A)**2+(BError/B)**2)


histogramName = "n_bjets"
# Find object by name
histogramInfoObject = 0
for i in histos:
    if i.m_name == histogramName:
        histogramInfoObject = i
        
        break
rebinHisto = histogramInfoObject.needsRebin()
print("Using the following object...\n",i) 
binRange = [1,1]

if CalculateRQCD:
    # Calculate RQCD
    dataSubtractedHistoCR = dataSubtract(histogramName,channelPath+CR,"Data",mcSamples,histogramInfoObject,rebin=rebinHisto)
    dataSubtractedHistoCRSS = dataSubtract(histogramName,channelPath+CRSS,"Data",mcSamples,histogramInfoObject,rebin=rebinHisto)
    makeNegativeBinsZero(dataSubtractedHistoCR)
    makeNegativeBinsZero(dataSubtractedHistoCRSS)
    RQCD = dataSubtractedHistoCR.Integral(binRange[0],binRange[1])/dataSubtractedHistoCRSS.Integral(binRange[0],binRange[1])
    RQCDError = errorAoverB(dataSubtractedHistoCR,dataSubtractedHistoCRSS,binRange)
    print("RQCD = ",RQCD," +- ",RQCDError)

Using the following object...
 Name: n_bjets, Bin Edges: [], Bin Steps: [], Bin Norm: 1.0, xTitle: n_bjets, Left Cut: 0, Right Cut: 1, Units: 


In [5]:
# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")


for hist in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = hist.needsRebin()
    
    print(hist.m_name,rebinHistogram)
    
    # Calculate RQCD in MJCRs
    if RQCDwithCRs:
        # Calculate RQCD
        dataSubtractedHistoCR = dataSubtract(hist.m_name,channelPath+CR,"Data",mcSamples,hist,rebin=rebinHistogram)
        dataSubtractedHistoCRSS = dataSubtract(hist.m_name,channelPath+CRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
        makeNegativeBinsZero(dataSubtractedHistoCR)
        makeNegativeBinsZero(dataSubtractedHistoCRSS)
        RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    # If not, use input value
    else:
        # RQCD =  1.26  +-  0.25 -> High mass region enhanced in MJ and loose jet cuts.
        # RQCD = 1.3 +- 0.25 -> Standard used in thesis version of the analysis.
        RQCD = 1.26#1.3*0.12
        UncerRQCD = 0.25#1.2*0.25
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(hist.m_name,channelPath+SRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    
    # If no MJ Evidence fix the bins that are in the final selection
    if totalMJisZero:
        SRhisto = mcAdd(hist.m_name,channelPath+SR,backgroundSamples,hist,rebin=rebinHistogram)
        cutLeft = hist.m_leftCut
        cutRight = hist.m_rightCut
        makeSRBinsConsistentWithNOMJ(MJ,cutLeft,cutRight,totalMJ,totalMJUncertainty,SRhisto)
        
    # Take into account the uncertainty in RQCD
    scaleUncertainty(MJ,UncerRQCD/RQCD)
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,hist.m_name)
    
    
multiJetFile.Close()

n_bjets False
Integral in SR bins: 6.698408126831055
lepiso False
Integral in SR bins: 6.698408126831055
n_jets_interval False
Integral in SR bins: 6.698408126831055
flavourJet1_basic_all False
Integral in SR bins: 6.698408996569924
flavourJet2_basic_all False
Integral in SR bins: 6.694606811623089
elecPdgID_basic_all False
Integral in SR bins: 2.923699832521379
muonPdgID_basic_all False
Integral in SR bins: 3.77215313911438
tauPdgID_basic_all False
Integral in SR bins: 6.698408888652921
nLightJets_basic_all False
Integral in SR bins: 6.698408603668213
tau_pt True
Integral in SR bins: 6.674287170171738
lep_pt True
Integral in SR bins: 6.698408825766819
delta_phi True
Integral in SR bins: 6.698408842086792
delta_y True
Integral in SR bins: 6.698408782482147
omega True
Integral in SR bins: 6.698408991098404
rnn_score_1p True
Integral in SR bins: 5.6593852043151855
rnn_score_3p True
Integral in SR bins: 1.0390238761901855
ljet0_pt True


/Users/user/Documents/HEP/VBF-Analysis/Scripts/histogramHelpers.py:276: RuntimeWarning: invalid value encountered in sqrt
  binError = totalMJUncertainty*np.sqrt(sf)


Integral in SR bins: 6.686482384800911
ljet1_pt True
Integral in SR bins: 6.698409142438322
pt_bal True
Integral in SR bins: 6.698409114032984
Z_centrality True
Integral in SR bins: 6.698408603668213
mass_jj True
Integral in SR bins: 6.672838568687439
reco_mass_i True
Integral in SR bins: 5.708888918161392
reco_mass_o True
Integral in SR bins: 0.869034961797297
reco_mass_ True
Integral in SR bins: 6.577923953533173
Z_pt_reco_i_basic_all True
Integral in SR bins: 5.813137095654383
Z_pt_reco_o_basic_all True
Integral in SR bins: 0.8783623097115196
ratio_zpt_sumjetpt_basic_all True
Integral in SR bins: 6.698408968979493
vec_sum_pt_jets_basic_all True
Integral in SR bins: 6.69840871123597
moreCentralJet_basic_all True
Integral in SR bins: 6.69840869680047
lessCentralJet_basic_all True
Integral in SR bins: 6.698408870724961
delta_phijj_basic_all True
Integral in SR bins: 6.698408842086792
massTauClosestJet_basic_all True
Integral in SR bins: 6.698408818338066
massLepClosestJet_basic_all Tru

Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 5 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 6 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of

In [22]:
from ctypes import *
osE = c_double(0.0)
ssE = c_double(0.0)
osHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCROS/","Data",mcSamples,histos,rebin=True)
ssHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCRSS/","Data",mcSamples,histos,rebin=True)
osIntegral = osHisto.IntegralAndError(1,-1,osE)
ssIntegral = ssHisto.IntegralAndError(1,-1,ssE)

print("OS: ",osIntegral)
print("SS: ",ssIntegral)

osE = osE.value
ssE = ssE.value

print("OS E: ",osE)
print("SS E: ",ssE)
print(osIntegral/ssIntegral," +- ",osIntegral/ssIntegral*np.sqrt((osE/osIntegral)**2+(ssE/ssIntegral)))

OS:  94.53706323355436
SS:  88.52085138997063
OS E:  19.725730909960834
SS E:  10.652698160888127
1.0679637819690624  +-  0.43233208949062885
